In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)
#Read in file
df = pd.read_csv("outputcities.csv")
df.head()


,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Busselton,-33.65,115.33,59.00,51.91,80,996,14.16,95
1,Rikitea,-23.12,-134.97,73.13,68.92,76,1016,16.93,87
2,Clyde River,70.47,-68.59,-0.40,-17.95,70,1016,19.46,20
3,Hilo,19.73,-155.09,79.09,78.55,62,1017,10.29,75
4,Atuona,-9.80,-139.03,77.27,76.98,72,1011,11.92,40


In [9]:
# Store latitude and longitude in locations
locations = df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidities = df["Humidity"].astype(float)
locations

,Latitude,Longitude
0,-33.65,115.33
1,-23.12,-134.97
2,70.47,-68.59
3,19.73,-155.09
4,-9.80,-139.03
...,...,...
557,54.43,31.90
558,26.06,78.21
559,50.68,36.55
560,35.53,-108.74


In [45]:
#Part 1 cities
coordinates = df[["Latitude", "Longitude"]]
idealCoordinates = df_sub[["Latitude", "Longitude"]]
humidities = df.Humidity

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

# # Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates, info_box_content=df.City)
# # Add the layer to the map
fig.add_layer(markers)

heat = gmaps.heatmap_layer(coordinates, weights = humidities, dissipating=False, max_intensity=500)
# Add the layer to the map
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [17]:
#Filter for cities between 71 and 79 degrees, wind speeds less than 10 mph, and zero cloudiness
mask = (df.Temperature < 80) & (df.Temperature > 70) & (df["Wind Speed"] < 10) & (df["Cloudiness"] == 0)
df_sub = df.loc[mask].reset_index(drop=True)
df_sub.head()

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Malakal,9.53,31.66,77.16,74.26,32,1011,3.09,0
1,Comarapa,-17.90,-64.48,72.64,68.85,45,1008,6.93,0
2,Kyaikkami,16.08,97.57,76.84,80.96,79,1011,6.06,0
3,Cap Malheureux,-19.98,57.61,73.00,77.99,81,1020,1.99,0
4,Sur,22.57,59.53,75.79,77.83,65,1018,4.41,0


In [22]:
#The ideal cities
df_sub.City.values

array(['Malakal', 'Comarapa', 'Kyaikkami', 'Cap Malheureux', 'Sur',
       'Verāval', 'Mount Isa', 'Salalah', 'Coyhaique', 'Safaga', 'Yanam',
       'Sittwe', 'Riyadh', 'Lano', 'Santa Cruz Cabrália'], dtype=object)

In [23]:
#PLot ideal cities
coordinates = df[["Latitude", "Longitude"]]
idealCoordinates = df_sub[["Latitude", "Longitude"]]
humidities = df.Humidity

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

# # Assign the marker layer to a variable
markers = gmaps.marker_layer(idealCoordinates, info_box_content=df_sub.City)
# # Add the layer to the map
fig.add_layer(markers)

heat = gmaps.heatmap_layer(coordinates, weights = humidities, dissipating=False, max_intensity=500)
# Add the layer to the map
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [39]:
#Pull Data for closets hotel

#Lists
names = []
addresses = []



# find the closest hotel

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "lodging",
    "key": g_key,
}

# use iterrows to iterate through dataframe
for index, row in df_sub.iterrows():
    
    location = f"{row.Latitude}, {row.Longitude}"

    # add keyword to params dict
    params['location'] = location

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row.City}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest Hotel is {results[0]['name']}.")
        

    #define          
        name = results[0]['name']
        address = results[0]['vicinity']
      

    #write to        
        names.append(name)
        addresses.append(address)
    

              
     #Error Loop   
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        names.append(np.nan)
        addresses.append(np.nan)

        
    print("------------")
              
df_sub["Hotel"] = names
df_sub["Address"] = addresses



Retrieving Results for Index 0: Malakal.
Closest Hotel is Nile Palace Hotel.
------------
Retrieving Results for Index 1: Comarapa.
Closest Hotel is Casa Hogar Manos De Amor.
------------
Retrieving Results for Index 2: Kyaikkami.
Closest Hotel is St. joseph catholic church.
------------
Retrieving Results for Index 3: Cap Malheureux.
Closest Hotel is Chez Lui.
------------
Retrieving Results for Index 4: Sur.
Closest Hotel is Arabian Sea Motel.
------------
Retrieving Results for Index 5: Verāval.
Closest Hotel is Jayesh fofandi.
------------
Retrieving Results for Index 6: Mount Isa.
Closest Hotel is Aboriginal Hostels Ltd..
------------
Retrieving Results for Index 7: Salalah.
Closest Hotel is Al Sahwa New Building.
------------
Retrieving Results for Index 8: Coyhaique.
Closest Hotel is HUEMUL PATAGÓN.
------------
Retrieving Results for Index 9: Safaga.
Closest Hotel is Joya resort.
------------
Retrieving Results for Index 10: Yanam.
Closest Hotel is Hotel Sarvainn.
------------


In [40]:
df_sub.head()

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness,hotel_name,address,Hotel,Address
0,Malakal,9.53,31.66,77.16,74.26,32,1011,3.09,0,GMB,"GatluakMutBech roads, GMB, Malakal",Nile Palace Hotel,Malakal
1,Comarapa,-17.90,-64.48,72.64,68.85,45,1008,6.93,0,K'ajllo,Comarapa,Casa Hogar Manos De Amor,Comarapa
2,Kyaikkami,16.08,97.57,76.84,80.96,79,1011,6.06,0,SAN Tea Shop,"Myoma Zay Road, Kyaikkhami",St. joseph catholic church,"Market Bus Stop, Kyaikkhami"
3,Cap Malheureux,-19.98,57.61,73.00,77.99,81,1020,1.99,0,SOKIUTE BY SOPHIE P,"Pointe l’Hortal, MU, Cap Malheureux",Chez Lui,"MU, Route Royale, Cap Malheureux"
4,Sur,22.57,59.53,75.79,77.83,65,1018,4.41,0,Arabian Sea Motel,"Aseelah - Al Sharqiyah, Al Ashkharah, صور",Arabian Sea Motel,"Aseelah - Al Sharqiyah, Al Ashkharah, صور"


In [41]:
hotel_info = []

for indx, row in df_sub.iterrows():
    info = f"City: {row.City}<br>-----------<br>Hotel: {row['Hotel']}<br>-----------<br>Address: {row.address}"
    hotel_info.append(info)

In [42]:
#Plot ideal cities with closest hotel
coordinates = df[["Latitude", "Longitude"]]
idealCoordinates = df_sub[["Latitude", "Longitude"]]
humidities = df.Humidity

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

# # Assign the marker layer to a variable
markers = gmaps.marker_layer(idealCoordinates, info_box_content=hotel_info)
# # Add the layer to the map
fig.add_layer(markers)

heat = gmaps.heatmap_layer(coordinates, weights = humidities, dissipating=False, max_intensity=500)
# Add the layer to the map
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…